In [1]:
#ui library
!pip install ipywidgets


In [2]:
#import all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import ipywidgets as widgets
from IPython.display import display, clear_output

sns.set_style("whitegrid")


In [4]:
#demo dataset
np.random.seed(42)

roles = ["Data Analyst","Data Scientist","ML Engineer","Business Analyst"]
locations = ["India","USA","UK","Germany"]
experience_levels = ["EN","MI","SE","EX"]

data = []

for _ in range(1000):
    role = np.random.choice(roles)
    location = np.random.choice(locations)
    exp = np.random.choice(experience_levels)
    skill_count = np.random.randint(2,8)
    
    base_salary = 40000
    
    exp_multiplier = {"EN":1,"MI":1.5,"SE":2,"EX":2.5}
    location_multiplier = {"India":1,"USA":2.5,"UK":2,"Germany":1.8}
    
    salary = base_salary * exp_multiplier[exp] * location_multiplier[location]
    salary += skill_count * 3000
    salary += np.random.randint(-5000,5000)
    
    data.append([role,location,exp,skill_count,int(salary)])

df = pd.DataFrame(data, columns=["Role","Location","Experience","Skill_Count","Salary"])

df.head()


,Role,Location,Experience,Skill_Count,Salary
0,ML Engineer,Germany,EN,4,84191
1,Data Analyst,India,SE,3,84466
2,ML Engineer,UK,SE,6,181322
3,Data Scientist,India,MI,5,76949
4,Data Scientist,Germany,EX,6,194184


In [6]:
#encode and train
le_role = LabelEncoder()
le_loc = LabelEncoder()
le_exp = LabelEncoder()

df["Role"] = le_role.fit_transform(df["Role"])
df["Location"] = le_loc.fit_transform(df["Location"])
df["Experience"] = le_exp.fit_transform(df["Experience"])

X = df.drop("Salary",axis=1)
y = df["Salary"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

model = RandomForestRegressor(n_estimators=200)
model.fit(X_train,y_train)

print("Model Ready ")


Model Ready 


In [7]:
#interactiv ui
role_dropdown = widgets.Dropdown(options=le_role.classes_, description="Role:")
location_dropdown = widgets.Dropdown(options=le_loc.classes_, description="Location:")
experience_dropdown = widgets.Dropdown(options=le_exp.classes_, description="Experience:")
skill_slider = widgets.IntSlider(value=3,min=1,max=10,description="Skill Count:")

button = widgets.Button(description="Predict Career Future 🚀")

output = widgets.Output()

def predict(b):
    with output:
        clear_output()
        
        role_encoded = le_role.transform([role_dropdown.value])[0]
        loc_encoded = le_loc.transform([location_dropdown.value])[0]
        exp_encoded = le_exp.transform([experience_dropdown.value])[0]
        
        input_data = np.array([[role_encoded,loc_encoded,exp_encoded,skill_slider.value]])
        predicted_salary = model.predict(input_data)[0]
        
        print(" Predicted Current Salary: $", int(predicted_salary))
        
        # 5-year projection
        years = np.arange(1,6)
        growth = 0.08
        future_salary = [predicted_salary*((1+growth)**year) for year in years]
        
        plt.figure()
        plt.plot(years,future_salary)
        plt.title("5-Year Salary Projection")
        plt.xlabel("Years")
        plt.ylabel("Projected Salary")
        plt.show()
        
        # Automation Risk
        risk_map = {"Data Analyst":0.45,"Data Scientist":0.30,"ML Engineer":0.25,"Business Analyst":0.50}
        risk = risk_map.get(role_dropdown.value,0.35)
        print("⚠ Automation Risk:",risk*100,"%")
        
button.on_click(predict)

display(role_dropdown,location_dropdown,experience_dropdown,skill_slider,button,output)


Dropdown(description='Role:', options=(np.int64(0), np.int64(1), np.int64(2), np.int64(3)), value=np.int64(0))

Dropdown(description='Location:', options=(np.int64(0), np.int64(1), np.int64(2), np.int64(3)), value=np.int64…

Dropdown(description='Experience:', options=(np.int64(0), np.int64(1), np.int64(2), np.int64(3)), value=np.int…

IntSlider(value=3, description='Skill Count:', max=10, min=1)

Button(description='Predict Career Future 🚀', style=ButtonStyle())

Output()